<b>June 16, 2025 • 10:30 AM</b>

# <b>Provider Segmentation Project</b>

<b>STATEMENT: Provider satisfaction has emerged as one of the most important factors that guarantee the success of service providing; it has been posited as a key stimulant of accept service, cancel service intentions and provider loyaty. A comprehensive review of the literature, theories and models have been carried out to propose the models for customer activation and customer retention. Five major factors that contributed to the success of a cleaning service have been identified as: service quality, information quality, trust and net benefit. The research furthermore investigated the factors that influence the service provider acceptance intention. The combination of both utilitarian value and hedonistic values are needed to affect the repeat service intention (loyalty) positively. The data is collected from our application and platforms. Results indicate the segmentation of service provider based on their behavior preference, which are very much critical for provider satisfaction.</b>

# <b>Importing Dataset</b>

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import pymysql as pydb
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine, text, Column, Integer, String, DateTime
from sqlalchemy.orm import sessionmaker, declarative_base
from IPython.display import display, clear_output

import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors
from scipy.stats import norm

import warnings
import matplotlib.ticker as ticker
warnings.filterwarnings("ignore")

from calendar import monthrange

## <b>Import Data from Database</b>
* Credential is required
* Batch Processing system

In [ ]:
load_dotenv()
db = os.getenv('BENEAT_DB')
engine = create_engine(db)

query = '''
SELECT o.professional_ID, o.id AS booking_id, 
       IFNULL(o.work_hour, 0) + IFNULL(oah.additional_hour, 0) AS total_work_hour,
       CASE WHEN o.select_receiving_service_date > 0 
       THEN o.select_receiving_service_date
       ELSE o.order_date END AS booking_date,
       
       o.cleaning_date, o.complete_cleaning_date, pr.rating, pb.report_problem_id,
       CASE WHEN o.cancel > 0 
       THEN 1 
       ELSE 0 END AS is_cancel,
       
       o.professional_income,
       YEAR(CASE WHEN o.select_receiving_service_date > 0 
            THEN o.select_receiving_service_date
            ELSE o.order_date END) AS booking_year,
       MONTH(CASE WHEN o.select_receiving_service_date > 0 
            THEN o.select_receiving_service_date 
            ELSE o.order_date END) AS booking_month,
       DAY(CASE WHEN o.select_receiving_service_date > 0 
            THEN o.select_receiving_service_date 
            ELSE o.order_date END) AS booking_day,
       YEAR(o.cleaning_date) AS cleaning_year,
       MONTH(o.cleaning_date) AS cleaning_month,
       DAY(o.cleaning_date) AS cleaning_day
       
FROM tb_order o

LEFT JOIN (SELECT order_id, SUM(work_hour) AS additional_hour
           FROM tb_order_additional_hours oah
           WHERE oah.status = 1 AND oah.cancel = 0
           GROUP BY order_id) oah 
ON o.id = oah.order_id

LEFT JOIN tb_professional_rating pr
ON o.id = pr.order_id

LEFT JOIN (SELECT id AS report_problem_id, order_id, professional_id
           FROM tb_report_problems) pb
ON o.id = pb.order_id

LEFT JOIN tb_professional p ON o.professional_id = p.id
WHERE o.paid_status = 1
AND o.status = 2
-- AND o.cancel = 0
AND o.professional_id NOT IN (0, 300)
AND o.service_request_id IS NULL
AND o.complete_cleaning_date < DATE(NOW())
AND o.professional_id > 10
'''
professional_booking_raw = pd.read_sql(sql=text(query), con=engine.connect())

professional_booking_raw

,professional_id,booking_id,total_work_hour,booking_date,cleaning_date,complete_cleaning_date,rating,report_problem_id,is_cancel,professional_income,booking_year,booking_month,booking_day,cleaning_year,cleaning_month,cleaning_day
0,11,38,2.0,2016-04-03 22:12:37,2016-04-11 14:00:00,2016-04-11 17:56:33,2.5,NaN,0,300.0,2016.0,4.0,3.0,2016,4,11
1,12,46,2.0,2016-04-09 16:19:17,2016-04-17 16:00:00,2016-04-17 18:02:05,NaN,NaN,0,300.0,2016.0,4.0,9.0,2016,4,17
2,12,49,2.0,2016-04-09 16:28:06,2016-04-11 09:30:00,2016-04-11 10:20:35,NaN,NaN,0,300.0,2016.0,4.0,9.0,2016,4,11
3,12,52,2.0,2016-04-09 16:39:55,2016-04-10 12:00:00,2016-04-10 14:17:06,NaN,NaN,0,300.0,2016.0,4.0,9.0,2016,4,10
4,12,54,2.0,2016-04-09 16:51:52,2016-04-18 16:00:00,2016-04-18 18:26:42,NaN,NaN,0,300.0,2016.0,4.0,9.0,2016,4,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068727,11965,1260254,2.0,2025-03-04 15:18:48,2025-03-04 18:30:00,2025-03-04 22:24:05,NaN,NaN,0,350.0,2025.0,3.0,4.0,2025,3,4
1068728,12191,1260258,3.0,2025-03-04 15:23:30,2025-03-04 18:30:00,2025-03-04 21:35:33,NaN,NaN,0,525.0,2025.0,3.0,4.0,2025,3,4
1068729,11166,1260266,3.0,2025-03-04 15:30:37,2025-03-04 18:30:00,2025-03-04 21:36:55,NaN,NaN,0,525.0,2025.0,3.0,4.0,2025,3,4
1068730,10757,1260295,3.0,2025-03-04 15:43:44,2025-03-04 18:30:00,2025-03-04 21:27:48,NaN,NaN,0,525.0,2025.0,3.0,4.0,2025,3,4


# <b>Data Preprocessing & EDA</b>

In [ ]:
from calendar import monthrange

# Copy the raw data
professional_booking_summary_monthly = professional_booking_raw.copy()

# Ensure date columns are in datetime format
professional_booking_summary_monthly['booking_date'] = pd.to_datetime(professional_booking_summary_monthly['booking_date'], errors='coerce')
professional_booking_summary_monthly['cleaning_date'] = pd.to_datetime(professional_booking_summary_monthly['cleaning_date'], errors='coerce')

# Extract year and month for both booking and cleaning dates
professional_booking_summary_monthly['booking_year'] = professional_booking_summary_monthly['booking_date'].dt.year
professional_booking_summary_monthly['booking_month'] = professional_booking_summary_monthly['booking_date'].dt.month
professional_booking_summary_monthly['cleaning_year'] = professional_booking_summary_monthly['cleaning_date'].dt.year
professional_booking_summary_monthly['cleaning_month'] = professional_booking_summary_monthly['cleaning_date'].dt.month

# Filter rows where cleaning year/month is greater than booking year/month
future_cleanings = professional_booking_summary_monthly[
    (professional_booking_summary_monthly['cleaning_year'] > professional_booking_summary_monthly['booking_year']) |
    (
        (professional_booking_summary_monthly['cleaning_year'] == professional_booking_summary_monthly['booking_year']) &
        (professional_booking_summary_monthly['cleaning_month'] > professional_booking_summary_monthly['booking_month'])
    )
]

# Group these filtered rows to calculate on_hand_booking
on_hand_booking_summary = future_cleanings.groupby(
    ['cleaning_year', 'cleaning_month', 'professional_id']
).agg(
    on_hand_booking=('booking_id', 'count')  # Count future bookings
).reset_index()

# Calculate the main summary for cleaning metrics
professional_booking_summary_monthly = professional_booking_summary_monthly.groupby(
    ['cleaning_year', 'cleaning_month', 'professional_id']
).agg(
    total_bookings=('booking_id', 'count'),  # Count total bookings in the cleaning month
    total_income=('professional_income', 'sum'),  # Sum income
    total_cancel = ('is_cancel', 'sum'),
    total_problem = ('report_problem_id', 'nunique'),
    avg_rating=('rating', 'mean'),  # Average rating
    total_work_hour=('total_work_hour', 'sum'),  # Sum work hours
    active_days=('cleaning_date', lambda x: x.dt.date.nunique())  # Unique active cleaning days
).reset_index()

# Merge on_hand_booking with the main summary
final_summary = professional_booking_summary_monthly.merge(
    on_hand_booking_summary,
    on=['cleaning_year', 'cleaning_month', 'professional_id'],
    how='left'
)

# Fill missing values in on_hand_booking
final_summary['on_hand_booking'] = final_summary['on_hand_booking'].fillna(0).astype(int)

# Sort the final summary
final_summary = final_summary.sort_values(['professional_id', 'cleaning_year', 'cleaning_month']).reset_index(drop=True)

def calculate_tenure_month(group):
    # Initialize variables
    tenure_months = []
    last_cleaning_month = None
    min_year, min_month = group.iloc[0]['cleaning_year'], group.iloc[0]['cleaning_month']
    active_month_counter = 1

    # Loop over each row to calculate tenure
    for _, row in group.iterrows():
        current_year, current_month = row['cleaning_year'], row['cleaning_month']

        # If there is a gap in months, reset the active month counter
        if last_cleaning_month is None or (current_year - last_cleaning_month[0]) * 12 + (current_month - last_cleaning_month[1]) > 1:
            active_month_counter = 1  # Reset on gap
        else:
            active_month_counter += 1

        # Append the active month counter
        tenure_months.append(active_month_counter)

        # Update the last cleaning month
        last_cleaning_month = (current_year, current_month)

    return tenure_months

final_summary['consecutive_active_month'] = final_summary.groupby('professional_id').apply(calculate_tenure_month).explode().astype(int).values

# Calculate consecutive_active_month
def calculate_consecutive_months(group):
    consecutive_active = 0
    consecutive_list = []
    for is_active in group['total_bookings'] > 0:  # Check if bookings exist
        if is_active:
            consecutive_active += 1
        else:
            consecutive_active = 0  # Reset on inactivity
        consecutive_list.append(consecutive_active)
    return consecutive_list

final_summary['active_month'] = final_summary.groupby('professional_id').apply(calculate_consecutive_months).explode().astype(int).values

# Calculate active_day_ratio
final_summary['days_in_month'] = final_summary.apply(
    lambda row: monthrange(int(row['cleaning_year']), int(row['cleaning_month']))[1],
    axis=1
)
final_summary['active_day_ratio'] = final_summary['active_days'] / final_summary['days_in_month']

# Drop the helper column 'days_in_month' if not needed
final_summary.drop(columns=['days_in_month'], inplace=True)

# Reset index for clean output
final_summary.reset_index(drop=True, inplace=True)

# Display the results
final_summary = final_summary.sort_values(['professional_id', 'cleaning_year', 'cleaning_month']).reset_index(drop=True)

final_summary


,cleaning_year,cleaning_month,professional_id,total_bookings,total_income,total_cancel,total_problem,avg_rating,total_work_hour,active_days,on_hand_booking,consecutive_active_month,active_month,active_day_ratio
0,2016,4,11,1,300.0,0,0,2.500000,2.0,1,0,1,1,0.033333
1,2016,4,12,20,6150.0,0,0,4.750000,40.5,13,0,1,1,0.433333
2,2016,5,12,25,8276.0,0,0,4.916667,57.5,18,3,2,2,0.580645
3,2016,6,12,11,2999.0,0,0,4.125000,21.0,7,2,3,3,0.233333
4,2018,3,12,2,650.0,0,0,4.000000,4.5,2,0,1,4,0.064516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74617,2025,3,12408,5,2275.0,0,0,5.000000,13.0,3,2,1,1,0.096774
74618,2025,3,12414,4,1278.0,0,0,NaN,7.0,2,1,1,1,0.064516
74619,2025,3,12417,2,700.0,0,0,NaN,5.0,2,0,1,1,0.064516
74620,2025,3,12421,1,350.0,0,0,NaN,2.0,1,0,1,1,0.032258


## <b>Latest Booking Result</b>

In [ ]:
latest_booking = professional_booking_raw.copy()

# Convert 'complete_cleaning_date' to datetime
latest_booking['complete_cleaning_date'] = pd.to_datetime(latest_booking['complete_cleaning_date'], errors='coerce')

# Find the latest booking for each professional_id
latest_booking = latest_booking.loc[
    latest_booking.groupby('professional_id')['complete_cleaning_date'].idxmax()
]

# Extract year, month, and day from the latest booking date
latest_booking['latest_booking_year'] = latest_booking['complete_cleaning_date'].dt.year
latest_booking['latest_booking_month'] = latest_booking['complete_cleaning_date'].dt.month
latest_booking['latest_booking_day'] = latest_booking['complete_cleaning_date'].dt.day

# Select the desired columns
latest_booking_result = latest_booking[['professional_id', 'complete_cleaning_date', 'latest_booking_year', 'latest_booking_month', 'latest_booking_day']]

# Rename the columns for clarity
latest_booking_result = latest_booking_result.rename(columns={'complete_cleaning_date': 'latest_booking'})

# Define the reference date (2024-12-31)
reference_date = pd.to_datetime('2025-03-01')

# Calculate inactive days as the difference between the reference date and the latest booking date
latest_booking_result['inactive_days'] = (reference_date - latest_booking_result['latest_booking']).dt.days


# Display the result
latest_booking_result.reset_index(drop=True, inplace=True)

latest_booking_result['is_churned'] = np.where(latest_booking_result['inactive_days'] > 90, 1, 0)

latest_booking_result = latest_booking_result[['professional_id', 'latest_booking_year', 'latest_booking_month', 'is_churned']]

latest_booking_result = latest_booking_result[latest_booking_result['is_churned'] == 1]

latest_booking_result

,professional_id,latest_booking_year,latest_booking_month,is_churned
0,11,2016,4,1
1,12,2018,4,1
2,16,2016,6,1
3,17,2016,7,1
4,18,2017,3,1
...,...,...,...,...
6685,11763,2024,11,1
6696,11778,2024,11,1
6700,11784,2024,11,1
6702,11786,2024,11,1


## <b>Service Provider Age</b>
* Some provider got confuse between A.D. 2025 and B.E. 2568

In [ ]:
load_dotenv()
db = os.getenv('BENEAT_DB')
engine = create_engine(db)

query = '''
SELECT id AS professional_id, 
    CASE WHEN TIMESTAMPDIFF(YEAR, CASE WHEN p.birthday 
                                  LIKE '%,%' 
                                  THEN DATE(CONVERT(p.birthday, CHAR) - 543) 
                                  ELSE p.birthday END, CURDATE()) < 18
           OR TIMESTAMPDIFF(YEAR, CASE WHEN p.birthday 
                                  LIKE '%,%' 
                                  THEN DATE(CONVERT(p.birthday, CHAR) - 543) 
                                  ELSE p.birthday END, CURDATE()) > 60
         THEN NULL
         ELSE TIMESTAMPDIFF(YEAR, CASE WHEN p.birthday 
                                  LIKE '%,%' 
                                  THEN DATE(CONVERT(p.birthday, CHAR) - 543) 
                                  ELSE p.birthday END, CURDATE()) END AS age,
province_id, training_center_id, created_at, p.layoff_date
FROM tb_professional p
'''
professional_profile_raw = pd.read_sql(sql=text(query), con=engine.connect())

professional_profile_raw

,professional_id,age,province_id,training_center_id,created_at,layoff_date
0,1,34.0,1,3.0,2016-01-13 12:00:00,2020-12-08 14:00:02
1,2,37.0,2,3.0,2016-02-03 20:00:00,2022-02-25 11:16:53
2,3,NaN,2,3.0,2021-05-05 17:00:00,2021-05-06 14:00:03
3,4,NaN,1,3.0,2016-02-04 18:00:30,2021-09-17 14:00:04
4,5,45.0,2,3.0,2016-02-04 18:00:30,NaT
...,...,...,...,...,...,...
12149,12466,32.0,1,NaN,2025-03-04 14:16:00,NaT
12150,12467,32.0,1,NaN,2025-03-04 14:20:00,NaT
12151,12468,32.0,1,NaN,2025-03-04 14:20:00,NaT
12152,12469,32.0,1,NaN,2025-03-04 14:20:00,NaT
